In [84]:
import pandas as pd

In [85]:
df = pd.read_csv('Elem_Voca.csv')
df.head()

,학년,출판사(저자),교육과정,레슨,단어,뜻,예문,예문 해석,영영풀이
0,초3,YBM(김),2022,1,be,NaN,NaN,NaN,NaN
1,초3,YBM(김),2022,1,everyone,NaN,NaN,NaN,NaN
2,초3,YBM(김),2022,1,goodbye,NaN,NaN,NaN,NaN
3,초3,YBM(김),2022,1,hello,NaN,NaN,NaN,NaN
4,초3,YBM(김),2022,1,I,NaN,NaN,NaN,NaN


# TEXT ID

In [86]:
id = pd.read_csv('../../../DB/Text_ID.csv')
id.columns
drop_col = ['구분','교과서 확보', '어휘 자료','function', 'structure', 'Issue']
id.drop(columns=drop_col, inplace=True)
id['학년 정보'] = id['학년 정보'].astype(str).replace('영어', '', regex=True)
id.rename(columns={
    '학년 정보': '학년',
    '개정 정보': '교육과정'
}, inplace=True)
display(id)

,textbook ID,학년,교육과정,출판사
0,301,초3,2015,천재 (함순애)
1,302,초3,2015,YBM (김혜리)
2,303,초3,2015,YBM (최은경)
3,304,초3,2015,동아 (박기화)
4,305,초3,2015,아이스크림
...,...,...,...,...
88,905,중3,2015,비상 (김진완)
89,906,중3,2015,천재 (이재영)
90,907,중3,2015,천재 (정사열)
91,908,중3,2015,YBM (박준언)


In [87]:
# 변환 함수 정의
def modify_publisher(name):
    if "(" in name and ")" in name:  # 저자 이름이 있는 경우만 처리
        parts = name.split("(")  # 괄호를 기준으로 분리
        publisher = parts[0].strip()  # 출판사 이름
        author_initial = parts[1][0]  # 저자의 첫 글자만 추출
        return f"{publisher}({author_initial})"
    return name  # 저자가 없는 경우 그대로 반환

# 데이터 변환 적용
id["출판사"] = id["출판사"].apply(modify_publisher)

In [88]:
id['출판사'] = id['출판사'].replace('비상(김)','비상')

In [89]:
id['출판사'].value_counts()

출판사
천재(함)      8
천재(이)      8
동아(윤)      8
YBM(김)     7
YBM(최)     6
아이스크림      6
미래엔        6
능률(김)      4
YBM(박)     4
동아(이)      4
천재(김)      4
동아(박)      4
능률(양)      3
비상         3
천재(정)      3
YBM(송)     3
시사(최)      2
대교(이)      2
금성(최)      1
다락원(강)     1
미래(최)      1
지학사(민)     1
천재(소)      1
미래엔(문)     1
비상교육(황)    1
지학사(송)     1
Name: count, dtype: int64

In [90]:
id.head()

,textbook ID,학년,교육과정,출판사
0,301,초3,2015,천재(함)
1,302,초3,2015,YBM(김)
2,303,초3,2015,YBM(최)
3,304,초3,2015,동아(박)
4,305,초3,2015,아이스크림


In [91]:
id.columns

Index(['textbook ID', '학년', '교육과정', '출판사'], dtype='object')

In [92]:
id['학년'].value_counts()

학년
중1     23
초3     14
초4     14
초5     12
초6     12
중2      9
중3      9
Name: count, dtype: int64

In [93]:
id['학년'] = id['학년'].str.strip()
id['학년'].value_counts()

학년
중1    23
초3    14
초4    14
초5    12
초6    12
중2     9
중3     9
Name: count, dtype: int64

In [94]:
filtered_data = id.loc[id['학년'].isin(['초3', '초4', '초5', '초6'])]
filtered_data.head()

,textbook ID,학년,교육과정,출판사
0,301,초3,2015,천재(함)
1,302,초3,2015,YBM(김)
2,303,초3,2015,YBM(최)
3,304,초3,2015,동아(박)
4,305,초3,2015,아이스크림


In [95]:
filtered_data = filtered_data.rename(columns={
    "출판사" : "출판사(저자)"
})

In [96]:
merged_data = df.merge(filtered_data, on=["학년", "출판사(저자)", "교육과정"], how="left")
merged_data = merged_data[["textbook ID"] + [col for col in merged_data.columns if col != "textbook ID"]]
merged_data['textbook ID'] = merged_data['textbook ID'].astype(int)

In [97]:
merged_data.head()

,textbook ID,학년,출판사(저자),교육과정,레슨,단어,뜻,예문,예문 해석,영영풀이
0,310,초3,YBM(김),2022,1,be,NaN,NaN,NaN,NaN
1,310,초3,YBM(김),2022,1,everyone,NaN,NaN,NaN,NaN
2,310,초3,YBM(김),2022,1,goodbye,NaN,NaN,NaN,NaN
3,310,초3,YBM(김),2022,1,hello,NaN,NaN,NaN,NaN
4,310,초3,YBM(김),2022,1,I,NaN,NaN,NaN,NaN


In [98]:
merged_data.to_csv('merged_data.csv', encoding='utf-8-sig')